# HW4 - Deep Learning Course
## Nima Samadi
## Problem 3
### based on [this](https://huggingface.co/HooshvareLab/bert-fa-base-uncased-sentiment-digikala)

In [ ]:
%mkdir /content/HW4
%cd /content/HW4
!wget 'https://drive.google.com/uc?export=download&id=1QL3qN0VHsMjuLnMl2wif_O5N6RydveIv&confirm=T' -O poets-dataset.zip
!unzip -q poets-dataset.zip
%rm poets-dataset.zip
!ls

/content/HW4
--2023-02-05 09:11:02--  https://drive.google.com/uc?export=download&id=1QL3qN0VHsMjuLnMl2wif_O5N6RydveIv&confirm=T
Resolving drive.google.com (drive.google.com)... 74.125.137.100, 74.125.137.139, 74.125.137.102, ...
Connecting to drive.google.com (drive.google.com)|74.125.137.100|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-0s-a8-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/d772av1v60jm5mt3cc441apuck2o2gnq/1675588200000/04834621180226443203/*/1QL3qN0VHsMjuLnMl2wif_O5N6RydveIv?e=download&uuid=d1bdd13f-cf72-43da-9839-eb4cdc8f375d [following]
--2023-02-05 09:11:03--  https://doc-0s-a8-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/d772av1v60jm5mt3cc441apuck2o2gnq/1675588200000/04834621180226443203/*/1QL3qN0VHsMjuLnMl2wif_O5N6RydveIv?e=download&uuid=d1bdd13f-cf72-43da-9839-eb4cdc8f375d
Resolving doc-0s-a8-docs.googleusercontent.com (doc-0s-a8-docs.googleusercontent.com)..

In [ ]:
!pip install -q transformers
!pip install -q hazm
!pip install -q clean-text[gpl]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 47.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 86.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.7/316.7 KB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 44.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.6/233.6 KB 19.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 KB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.4/175.4 KB 11.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 KB 20.8 MB/s eta 0:00:00


## Dataset loading and preperation:

In [ ]:
%cd /content/HW4/poets-dataset

/content/HW4/poets-dataset


In [ ]:
import pandas as pd
import os
import numpy as np
import hazm
from tqdm.notebook import tqdm
import plotly.express as px
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
import plotly.graph_objects as go
import collections

In [ ]:
file_names = os.listdir()

In [ ]:
def readData(file_name):
  poet_name = file_name.split(".")[0]
  data = pd.read_csv(file_name, sep="\n")
  data.drop(0, axis=0, inplace=True)
  label = [poet_name]*data.shape[0]
  data['label'] = label
  data.rename(columns={data.columns[0]: 'poem'}, inplace=True)
  return data

In [ ]:
df = pd.DataFrame()
for file_name in file_names:
  print(f"Reading {file_name}")
  data = readData(file_name)
  df = pd.concat([df, data])

Reading shariar.txt
Reading rahi.txt
Reading bahaee.txt
Reading roodaki.txt
Reading gilani.txt
Reading babaafzal.txt
Reading razi.txt
Reading khayyam.txt
Reading obeyd.txt
Reading zahir.txt


In [ ]:
df['poem_len_by_words'] = df['poem'].apply(lambda t: len(hazm.word_tokenize(t)))
df

,poem,label,poem_len_by_words
1,مگر ره گم کند کو را گذار افتد به ما یارب,shariar,11
2,فراوان کن گذار آن مه گم کرده راه اینجا,shariar,9
3,کله جا ماندش این جا و نیامد دیگرش از پی,shariar,10
4,نیاید فی المثل آری گرش افتد کلاه اینجا,shariar,8
5,نگویم جمله با من باش و ترک کامکاران کن,shariar,9
...,...,...,...
7291,گفتی که به علم مرده را زنده کند,zahir,8
7292,عیسی نکند آنچ تو خر می گویی,zahir,7
7293,ای روی تو از لطافت آیینه روح,zahir,7
7294,خواهم که قدح های خیالت به صبوح,zahir,6


In [ ]:
min_max_len = df['poem_len_by_words'].min(), df['poem_len_by_words'].max()
print(f'Min: {min_max_len[0]} \tMax: {min_max_len[1]}')

Min: 2 	Max: 14


In [ ]:
fig = go.Figure()

fig.add_trace(go.Histogram(
    x=df['poem_len_by_words']
))

fig.update_layout(
    title_text='Distribution of word counts within peoms',
    xaxis_title_text='Word Count',
    yaxis_title_text='Frequency',
    bargap=0.2,
    bargroupgap=0.2)

fig.show()

In [ ]:
# removing poems that are out of [3, 11] range
MIN_POEM_LEN = 3
MAX_POEM_LEN = 11
df['poem_len_by_words'] = df['poem_len_by_words'].apply(lambda len_t: len_t if MIN_POEM_LEN <= len_t <= MAX_POEM_LEN else None)
df = df.dropna(subset=['poem_len_by_words'])
df = df.reset_index(drop=True)

In [ ]:
df

,poem,label,poem_len_by_words
0,مگر ره گم کند کو را گذار افتد به ما یارب,shariar,11.0
1,فراوان کن گذار آن مه گم کرده راه اینجا,shariar,9.0
2,کله جا ماندش این جا و نیامد دیگرش از پی,shariar,10.0
3,نیاید فی المثل آری گرش افتد کلاه اینجا,shariar,8.0
4,نگویم جمله با من باش و ترک کامکاران کن,shariar,9.0
...,...,...,...
29542,گفتی که به علم مرده را زنده کند,zahir,8.0
29543,عیسی نکند آنچ تو خر می گویی,zahir,7.0
29544,ای روی تو از لطافت آیینه روح,zahir,7.0
29545,خواهم که قدح های خیالت به صبوح,zahir,6.0


## Train and test split:

In [ ]:
labels = df['label'].unique()
df['label'] = df['label'].apply(lambda t: np.where(labels == t)[0].item())

In [ ]:
train, test = train_test_split(df, test_size=0.1, random_state=1, stratify=df['label'])

train_dataset = train.reset_index(drop=True)
test_dataset = test.reset_index(drop=True)

x_train, y_train = train_dataset['poem'].values, train_dataset['label'].values
x_test, y_test = test_dataset['poem'].values, test_dataset['label'].values

In [ ]:
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

(26592,) (26592,)
(2955,) (2955,)


## A) Transfer learning

In [ ]:
!pip install torch-summary

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from transformers import BertConfig, BertTokenizer
from transformers import BertModel

from transformers import AdamW
from transformers import get_linear_schedule_with_warmup

import torch
import torch.nn as nn
import torch.nn.functional as F
from torchsummary import summary

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f'device: {device}')

train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')

device: cpu
CUDA is not available.  Training on CPU ...


In [ ]:
TRAIN_BATCH_SIZE = 16
TEST_BATCH_SIZE = 16
EPOCHS = 3
LEARNING_RATE = 2e-5

MODEL_NAME_PATH = 'HooshvareLab/bert-fa-base-uncased'
OUTPUT_PATH = '/content/HW4/P3_model_transfer.bin'

In [ ]:
label2id = {label: i for i, label in enumerate(labels)}
id2label = {v: k for k, v in label2id.items()}

In [ ]:
tokenizer = BertTokenizer.from_pretrained(MODEL_NAME_PATH)
config = BertConfig.from_pretrained(MODEL_NAME_PATH, **{'label2id': label2id, 'id2label': id2label})

print(config.to_json_string())

{
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "shariar",
    "1": "rahi",
    "2": "bahaee",
    "3": "roodaki",
    "4": "gilani",
    "5": "babaafzal",
    "6": "razi",
    "7": "khayyam",
    "8": "obeyd",
    "9": "zahir"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "babaafzal": 5,
    "bahaee": 2,
    "gilani": 4,
    "khayyam": 7,
    "obeyd": 8,
    "rahi": 1,
    "razi": 6,
    "roodaki": 3,
    "shariar": 0,
    "zahir": 9
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.26.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 100000
}



In [ ]:
class PoemDataset(torch.utils.data.Dataset):
  def __init__(self, tokenizer, poems, labels, max_len=16):
    self.poems = poems
    self.labels = labels

    self.tokenizer = tokenizer
    self.max_len = max_len
  
  def __len__(self):
    return len(self.poems)

  def __getitem__(self, item):
    poem = str(self.poems[item])
    
    encoding = self.tokenizer.encode_plus(
        poem,
        add_special_tokens=True,
        truncation=True,
        max_length=self.max_len,
        return_token_type_ids=True,
        padding='max_length',
        return_attention_mask=True,
        return_tensors='pt'
    )
    inputs = {
        'poem': poem,
        'input_ids': encoding['input_ids'].flatten(),
        'attention_mask': encoding['attention_mask'].flatten(),
        'token_type_ids': encoding['token_type_ids'].flatten(),
        'label': torch.tensor(self.labels[item], dtype=torch.long)
    }

    return inputs
  
def create_data_loader(x, y, tokenizer, max_len, batch_size):
  dataset = PoemDataset(poems=x,
                        labels=y,
                        tokenizer=tokenizer,
                        max_len=max_len)
  return torch.utils.data.DataLoader(dataset, batch_size=batch_size)

In [ ]:
train_dataloader = create_data_loader(x_train, y_train, tokenizer, MAX_POEM_LEN, TRAIN_BATCH_SIZE)
test_dataloader = create_data_loader(x_test, y_test, tokenizer, MAX_POEM_LEN, TEST_BATCH_SIZE)

In [ ]:
class SentimentModelTransfer(nn.Module):
  def __init__(self, config):
    super(SentimentModelTransfer, self).__init__()

    self.bert = BertModel.from_pretrained(MODEL_NAME_PATH)      
    self.dropout = nn.Dropout(config.hidden_dropout_prob)
    self.classifier = nn.Linear(config.hidden_size, config.num_labels)

    for param in self.bert.parameters():
      param.requires_grad = False

  def forward(self, input_ids, attention_mask, token_type_ids):
    output = self.bert(input_ids=input_ids,
                       attention_mask=attention_mask,
                       token_type_ids=token_type_ids)
    pooled_output = self.dropout(output['pooler_output'])
    return self.classifier(pooled_output)

In [ ]:
model_transfer = SentimentModelTransfer(config=config).to(device)

Some weights of the model checkpoint at HooshvareLab/bert-fa-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
sample_data = next(iter(train_dataloader))
summary(model_transfer, (sample_data['input_ids'], sample_data['attention_mask'], sample_data['token_type_ids']))

Layer (type:depth-idx)                   Output Shape              Param #
├─BertModel: 1-1                         [[-1, 768]]               --
|    └─BertEmbeddings: 2-1               [-1, 11, 768]             --
|    |    └─Embedding: 3-1               [-1, 11, 768]             (76,800,000)
|    |    └─Embedding: 3-2               [-1, 11, 768]             (1,536)
|    |    └─Embedding: 3-3               [-1, 11, 768]             (393,216)
|    |    └─LayerNorm: 3-4               [-1, 11, 768]             (1,536)
|    |    └─Dropout: 3-5                 [-1, 11, 768]             --
|    └─BertEncoder: 2-2                  [[-1, 11, 768]]           --
|    └─BertPooler: 2-3                   [-1, 768]                 --
|    |    └─Linear: 3-6                  [-1, 768]                 (590,592)
|    |    └─Tanh: 3-7                    [-1, 768]                 --
├─Dropout: 1-2                           [-1, 768]                 --
├─Linear: 1-3                            [-1, 10]  

Layer (type:depth-idx)                   Output Shape              Param #
├─BertModel: 1-1                         [[-1, 768]]               --
|    └─BertEmbeddings: 2-1               [-1, 11, 768]             --
|    |    └─Embedding: 3-1               [-1, 11, 768]             (76,800,000)
|    |    └─Embedding: 3-2               [-1, 11, 768]             (1,536)
|    |    └─Embedding: 3-3               [-1, 11, 768]             (393,216)
|    |    └─LayerNorm: 3-4               [-1, 11, 768]             (1,536)
|    |    └─Dropout: 3-5                 [-1, 11, 768]             --
|    └─BertEncoder: 2-2                  [[-1, 11, 768]]           --
|    └─BertPooler: 2-3                   [-1, 768]                 --
|    |    └─Linear: 3-6                  [-1, 768]                 (590,592)
|    |    └─Tanh: 3-7                    [-1, 768]                 --
├─Dropout: 1-2                           [-1, 768]                 --
├─Linear: 1-3                            [-1, 10]  

In [ ]:
def simple_accuracy(y_true, y_pred):
  return (y_true == y_pred).mean()

def acc_and_f1(y_true, y_pred, average='weighted'):
  acc = simple_accuracy(y_true, y_pred)
  f1 = f1_score(y_true=y_true, y_pred=y_pred, average=average)
  return {
      "acc": acc,
      "f1": f1,
  }

def y_loss(y_true, y_pred, losses):
  y_true = torch.stack(y_true).cpu().detach().numpy()
  y_pred = torch.stack(y_pred).cpu().detach().numpy()
  y = [y_true, y_pred]
  loss = np.mean(losses)

  return y, loss

In [ ]:
def eval_op(model, data_loader, loss_fn):
  model.eval()

  losses = []
  y_pred = []
  y_true = []

  with torch.no_grad():
      for dl in tqdm(data_loader, total=len(data_loader), desc="Evaluation... "):
          
          input_ids = dl['input_ids']
          attention_mask = dl['attention_mask']
          token_type_ids = dl['token_type_ids']
          labels = dl['label']

          input_ids = input_ids.to(device)
          attention_mask = attention_mask.to(device)
          token_type_ids = token_type_ids.to(device)
          labelss = labels.to(device)

          outputs = model(
              input_ids=input_ids,
              attention_mask=attention_mask,
              token_type_ids=token_type_ids)
          
          _, preds = torch.max(outputs, dim=1)

          loss = loss_fn(outputs, labels)

          losses.append(loss.item())

          y_pred.extend(preds)
          y_true.extend(labels)
  
  eval_y, eval_loss = y_loss(y_true, y_pred, losses)
  return eval_y, eval_loss

In [ ]:
def train_op(model, 
             data_loader, 
             loss_fn, 
             optimizer, 
             scheduler, 
             step=0, 
             print_every_step=100, 
             eval=False,
             eval_cb=None,
             eval_loss_min=np.Inf,
             eval_data_loader=None, 
             clip=0.0):
    
    model.train()

    losses = []
    y_pred = []
    y_true = []

    for dl in tqdm(data_loader, total=len(data_loader), desc="Training... "):
        step += 1

        input_ids = dl['input_ids']
        attention_mask = dl['attention_mask']
        token_type_ids = dl['token_type_ids']
        labels = dl['label']

        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        token_type_ids = token_type_ids.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids)
        
        _, preds = torch.max(outputs, dim=1)

        loss = loss_fn(outputs, labels)

        losses.append(loss.item())

        loss.backward()

        if clip > 0.0:
            nn.utils.clip_grad_norm_(model.parameters(), max_norm=clip)
        optimizer.step()
        scheduler.step()

        y_pred.extend(preds)
        y_true.extend(labels)

        if eval:
            train_y, train_loss = y_loss(y_true, y_pred, losses)
            train_score = acc_and_f1(train_y[0], train_y[1], average='weighted')

            if step % print_every_step == 0:
                eval_y, eval_loss = eval_op(model, eval_data_loader, loss_fn)
                eval_score = acc_and_f1(eval_y[0], eval_y[1], average='weighted')

                if hasattr(eval_cb, '__call__'):
                    eval_loss_min = eval_cb(model, step, train_score, train_loss, eval_score, eval_loss, eval_loss_min)

    train_y, train_loss = y_loss(y_true, y_pred, losses)

    return train_y, train_loss, step, eval_loss_min

In [ ]:
optimizer = AdamW(model_transfer.parameters(), lr=1e-4, correct_bias=False)
total_steps = len(train_dataloader) * EPOCHS
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=total_steps
)

loss_fn = nn.CrossEntropyLoss()

step = 0
eval_loss_min = np.Inf
history = collections.defaultdict(list)

def eval_callback(epoch, epochs, output_path):
    def eval_cb(model, step, train_score, train_loss, eval_score, eval_loss, eval_loss_min):
        statement = ''
        statement += 'Epoch: {}/{}...'.format(epoch, epochs)
        statement += 'Step: {}...'.format(step)
        
        statement += 'Train Loss: {:.6f}...'.format(train_loss)
        statement += 'Train Acc: {:.3f}...'.format(train_score['acc'])

        statement += 'Valid Loss: {:.6f}...'.format(eval_loss)
        statement += 'Valid Acc: {:.3f}...'.format(eval_score['acc'])

        print(statement)

        if eval_loss <= eval_loss_min:
            print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
                eval_loss_min,
                eval_loss))
            
            torch.save(model.state_dict(), output_path)
            eval_loss_min = eval_loss
        
        return eval_loss_min


    return eval_cb


for epoch in tqdm(range(1, EPOCHS + 1), desc="Epochs... "):
    train_y, train_loss, step, eval_loss_min = train_op(
        model=model_transfer, 
        data_loader=train_dataloader, 
        loss_fn=loss_fn, 
        optimizer=optimizer, 
        scheduler=scheduler, 
        step=step, 
        print_every_step=250, 
        eval=True,
        eval_cb=eval_callback(epoch, EPOCHS, OUTPUT_PATH),
        eval_loss_min=eval_loss_min,
        eval_data_loader=test_dataloader, 
        clip=0.0)
    
    train_score = acc_and_f1(train_y[0], train_y[1], average='weighted')
    
    eval_y, eval_loss = eval_op(
        model=model_transfer, 
        data_loader=test_dataloader, 
        loss_fn=loss_fn)
    
    eval_score = acc_and_f1(eval_y[0], eval_y[1], average='weighted')
    
    history['train_acc'].append(train_score['acc'])
    history['train_loss'].append(train_loss)
    history['val_acc'].append(eval_score['acc'])
    history['val_loss'].append(eval_loss)

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning:

This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning



Epochs... :   0%|          | 0/3 [00:00<?, ?it/s]

Training... :   0%|          | 0/1662 [00:00<?, ?it/s]

Evaluation... :   0%|          | 0/185 [00:00<?, ?it/s]

Epoch: 1/3...Step: 250...Train Loss: 1.916058...Train Acc: 0.315...Valid Loss: 1.895263...Valid Acc: 0.320...
Validation loss decreased (inf --> 1.895263).  Saving model ...


...

Evaluation... :   0%|          | 0/185 [00:00<?, ?it/s]

Epoch: 3/3...Step: 4500...Train Loss: 1.845878...Train Acc: 0.341...Valid Loss: 1.880610...Valid Acc: 0.334...


Evaluation... :   0%|          | 0/185 [00:00<?, ?it/s]

Epoch: 3/3...Step: 4750...Train Loss: 1.851424...Train Acc: 0.339...Valid Loss: 1.879859...Valid Acc: 0.328...
Validation loss decreased (1.880066 --> 1.879859).  Saving model ...


Evaluation... :   0%|          | 0/185 [00:00<?, ?it/s]

In [ ]:
def predict(model, tokenizer, max_len=128, batch_size=32):
    test_dataloader = create_data_loader(x_test, y_test, tokenizer, MAX_POEM_LEN, TEST_BATCH_SIZE)

    predictions = []
    prediction_probs = []

    model.eval()
    with torch.no_grad():
        for dl in tqdm(test_dataloader, position=0):
            input_ids = dl['input_ids']
            attention_mask = dl['attention_mask']
            token_type_ids = dl['token_type_ids']

            # move tensors to GPU if CUDA is available
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            token_type_ids = token_type_ids.to(device)
            
            # compute predicted outputs by passing inputs to the model
            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                token_type_ids=token_type_ids)
            
            # convert output probabilities to predicted class
            _, preds = torch.max(outputs, dim=1)

            predictions.extend(preds)
            prediction_probs.extend(F.softmax(outputs, dim=1))

    predictions = torch.stack(predictions).cpu().detach().numpy()
    prediction_probs = torch.stack(prediction_probs).cpu().detach().numpy()

    return predictions, prediction_probs

In [ ]:
preds, probs = predict(model_transfer, tokenizer, max_len=MAX_POEM_LEN)

  0%|          | 0/185 [00:00<?, ?it/s]

In [ ]:
print(f'F1: {f1_score(preds, y_test, average="weighted")}')

F1: 0.09270313214145269


In [ ]:
print(classification_report(y_test, preds, target_names=labels))

              precision    recall  f1-score   support

     shariar       0.06      0.00      0.01       379
        rahi       0.35      0.20      0.25       250
      bahaee       0.31      0.11      0.16       257
     roodaki       0.00      0.00      0.00       208
      gilani       0.04      0.53      0.08       127
   babaafzal       0.04      0.05      0.04        88
        razi       0.09      0.24      0.13       228
     khayyam       0.03      0.09      0.04       128
       obeyd       0.12      0.00      0.01       561
       zahir       0.19      0.02      0.03       729

    accuracy                           0.08      2955
   macro avg       0.12      0.12      0.08      2955
weighted avg       0.15      0.08      0.06      2955



## B) Fine tune model
Due to limitations in training of whole model, I only trained model with Adam. However, quite similar results will be achieved in this scenario.

In [ ]:
TRAIN_BATCH_SIZE = 16
TEST_BATCH_SIZE = 16
EPOCHS = 2
LEARNING_RATE = 2e-5

MODEL_NAME_PATH = 'HooshvareLab/bert-fa-base-uncased'
OUTPUT_PATH = '/content/HW4/P3_model.bin'

In [ ]:
class SentimentModel(nn.Module):
  def __init__(self, config):
    super(SentimentModel, self).__init__()

    self.bert = BertModel.from_pretrained(MODEL_NAME_PATH)
    self.dropout = nn.Dropout(config.hidden_dropout_prob)
    self.classifier = nn.Linear(config.hidden_size, config.num_labels)

  def forward(self, input_ids, attention_mask, token_type_ids):
    output = self.bert(input_ids=input_ids,
                       attention_mask=attention_mask,
                       token_type_ids=token_type_ids)
    pooled_output = self.dropout(output['pooler_output'])
    return self.classifier(pooled_output)

In [ ]:
model = SentimentModel(config=config).to(device)

Some weights of the model checkpoint at HooshvareLab/bert-fa-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
optimizer = AdamW(model.parameters(), lr=LEARNING_RATE, correct_bias=False)
total_steps = len(train_dataloader) * EPOCHS
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=total_steps
)

loss_fn = nn.CrossEntropyLoss()

step = 0
eval_loss_min = np.Inf
history = collections.defaultdict(list)

def eval_callback(epoch, epochs, output_path):
    def eval_cb(model, step, train_score, train_loss, eval_score, eval_loss, eval_loss_min):
        statement = ''
        statement += 'Epoch: {}/{}...'.format(epoch, epochs)
        statement += 'Step: {}...'.format(step)
        
        statement += 'Train Loss: {:.6f}...'.format(train_loss)
        statement += 'Train Acc: {:.3f}...'.format(train_score['acc'])

        statement += 'Valid Loss: {:.6f}...'.format(eval_loss)
        statement += 'Valid Acc: {:.3f}...'.format(eval_score['acc'])

        print(statement)

        if eval_loss <= eval_loss_min:
            print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
                eval_loss_min,
                eval_loss))
            
            torch.save(model.state_dict(), output_path)
            eval_loss_min = eval_loss
        
        return eval_loss_min


    return eval_cb


for epoch in tqdm(range(1, EPOCHS + 1), desc="Epochs... "):
    train_y, train_loss, step, eval_loss_min = train_op(
        model=model, 
        data_loader=train_dataloader, 
        loss_fn=loss_fn, 
        optimizer=optimizer, 
        scheduler=scheduler, 
        step=step, 
        print_every_step=100, 
        eval=True,
        eval_cb=eval_callback(epoch, EPOCHS, OUTPUT_PATH),
        eval_loss_min=eval_loss_min,
        eval_data_loader=test_dataloader, 
        clip=0.0)
    
    train_score = acc_and_f1(train_y[0], train_y[1], average='weighted')
    
    eval_y, eval_loss = eval_op(
        model=model, 
        data_loader=test_dataloader, 
        loss_fn=loss_fn)
    
    eval_score = acc_and_f1(eval_y[0], eval_y[1], average='weighted')
    
    history['train_acc'].append(train_score['acc'])
    history['train_loss'].append(train_loss)
    history['val_acc'].append(eval_score['acc'])
    history['val_loss'].append(eval_loss)

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning:

This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning



Epochs... :   0%|          | 0/2 [00:00<?, ?it/s]

Training... :   0%|          | 0/1662 [00:00<?, ?it/s]

Evaluation... :   0%|          | 0/185 [00:00<?, ?it/s]

Epoch: 1/2...Step: 100...Train Loss: 2.107652...Train Acc: 0.249...Valid Loss: 2.033756...Valid Acc: 0.314...
Validation loss decreased (inf --> 2.033756).  Saving model ...


...

Evaluation... :   0%|          | 0/185 [00:00<?, ?it/s]

Epoch: 2/2...Step: 3200...Train Loss: 0.824578...Train Acc: 0.733...Valid Loss: 1.423717...Valid Acc: 0.535...


Evaluation... :   0%|          | 0/185 [00:00<?, ?it/s]

Epoch: 2/2...Step: 3300...Train Loss: 0.819275...Train Acc: 0.734...Valid Loss: 1.422436...Valid Acc: 0.535...


Evaluation... :   0%|          | 0/185 [00:00<?, ?it/s]

In [ ]:
preds, probs = predict(model, tokenizer, max_len=MAX_POEM_LEN)

  0%|          | 0/185 [00:00<?, ?it/s]

In [ ]:
print(f'F1: {f1_score(preds, y_test, average="weighted")}')

F1: 0.0437279382710288


In [ ]:
print(classification_report(preds, y_test, target_names=labels))

              precision    recall  f1-score   support

     shariar       0.01      0.01      0.01       262
        rahi       0.02      0.01      0.01       750
      bahaee       0.01      0.01      0.01       140
     roodaki       0.01      0.01      0.01       236
      gilani       0.72      0.81      0.77       113
   babaafzal       0.07      0.01      0.02       567
        razi       0.04      0.13      0.06        67
     khayyam       0.02      0.01      0.01       214
       obeyd       0.02      0.02      0.02       405
       zahir       0.01      0.05      0.02       201

    accuracy                           0.05      2955
   macro avg       0.09      0.11      0.09      2955
weighted avg       0.05      0.05      0.04      2955

